# BIGQUERY TO SNOWFLAKE SCHEMA CONVERSION 

### 1. CREATE SNOWFLAKE DDL SCRIPT FROM JSON SCHEMA FILES
### 2. EXECUTE SCRIPTS TO CREATE TABLES
### 3. PUT DATA FILES INTO INTERNAL STAGE
### 4. COPY DATA FILES FROM INTERNAL STAGE TO SNOWFLAKE TABLES
### 5. VALIDATE SUCCESS AND FAILED LOADS

In [2]:
# Import required libraries
from snowflake.snowpark.session import Session
from snowflake.snowpark.functions import avg, sum, col,lit
from snowflake.snowpark.functions import udf, sproc, col
from snowflake.snowpark.types import IntegerType, FloatType, LongType, DoubleType, DecimalType,StringType, BooleanType, Variant
from snowflake.snowpark.types import PandasSeries, PandasDataFrame
from snowflake.snowpark import functions as fn

import sys ,json
import io
import logging
import pandas as pd

import joblib
import pandas as pd
import numpy as np
import json

from snowflake.snowpark import version
print (f"snowflake snowpark version is: {version.VERSION}")

snowflake snowpark version is: (1, 0, 0)


### Install glob2 if not installed already

In [3]:
#!pip install glob2

### Create the scripts from the provided schema json files

In [4]:
script = []
from glob2 import glob
scriptfiles = glob('schema_files/*.json')
for j in scriptfiles:
# j = 'schema_files/affinity_api_weekly_dma_brand_channel_panel_schema.json'
    try:
        table = j.split('/')[-1].split('.')[0].replace('_schema','')
        f = open(j)
        data = json.load(f)
        col = []
        table_create = [f"create or replace table {table} ("]
        for i in range(len(data)):
            dtype = data[i]['type']
            if data[i]['type'] == 'BIGNUMERIC': ##not supported in snowflake
                dtype = 'NUMBER'
            if i<len(data)-1:
                table_create+=[data[i]['name']+' '+dtype]+[',']
            else:
                table_create+=[data[i]['name']+' '+dtype]        
        table_create+=[')']    
        script+=[''.join([i for i in table_create])]
    except:
        print (f"cannot create dml script for '{j}' , please check if valid file")
print (f"total files read : {len(scriptfiles)}")
print (f"total scripts generated : {len(script)}")
print (f"invalid files : {len(scriptfiles) - len(script)}")

cannot create dml script for 'schema_files/permission-tester.json' , please check if valid file
total files read : 44
total scripts generated : 43
invalid files : 1


### Connect to the snowflake session

In [5]:
snowflake_connection_cfg = open('cred.json')
snowflake_connection_cfg = snowflake_connection_cfg.read()
snowflake_connection_cfg = json.loads(snowflake_connection_cfg)

# Creating Snowpark Session
load_session = Session.builder.configs(snowflake_connection_cfg).create()
print('Current Database:', load_session.get_current_database())
print('Current Schema:', load_session.get_current_schema())
print('Current Warehouse:', load_session.get_current_warehouse())
print("Warehouse set up:")
load_session.sql("show warehouses like 'APP_WH'").collect()

Current Database: "IIC_POC_DB"
Current Schema: "PUBLIC"
Current Warehouse: "APP_WH"
Warehouse set up:


[Row(name='APP_WH', state='STARTED', type='STANDARD', size='X-Small', min_cluster_count=1, max_cluster_count=1, started_clusters=1, running=0, queued=0, is_default='N', is_current='Y', auto_suspend=600, auto_resume='true', available=' 100', provisioning='0', quiescing='0', other='0', created_on=datetime.datetime(2022, 2, 27, 4, 51, 57, 85000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), resumed_on=datetime.datetime(2023, 1, 2, 20, 53, 46, 463000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), updated_on=datetime.datetime(2023, 1, 2, 20, 53, 46, 463000, tzinfo=<DstTzInfo 'America/Los_Angeles' PST-1 day, 16:00:00 STD>), owner='SYSADMIN', comment='', enable_query_acceleration='false', query_acceleration_max_scale_factor=8, resource_monitor='null', actives=1, pendings=0, failed=0, suspended=0, uuid='1463550724', scaling_policy='STANDARD')]

### Create tables in snowflake

In [7]:
for s in script:
    try:
        load_session.sql(s).collect()
    except:
        print (f"cannot create table for dml script , please check if script is valid!")

### PUT the data files from your local folder into snowflake internal stage

In [ ]:
load_session.sql("CREATE OR REPLACE STAGE stage_data").collect()

In [9]:
datafiles = glob('Synthetic data/*.csv')

In [ ]:
for c in datafiles:
    try:
        load_session.file.put(c, 'stage_data')
    except:
        print (f"cannot load file {c}")
print (f"total number of data file available : {len(datafiles)}")

In [ ]:
internal_stage_list = load_session.sql("list @stage_data").collect()
print (f"total number of data file available : {len(internal_stage_list)}")

### Load Data to Snowflake from internal stage

In [10]:
load_script = []
loads_failed = []
loads_success = []
stage_directory = 'stage_data'
for d in datafiles:
    copy = f"copy into {d.split('/')[-1].split('.')[0].replace('synthetic_data_','')}\
     from @{stage_directory}/{d.split('/')[-1].split('.')[0]}.csv.gz "
    form = '''FILE_FORMAT = (TYPE = 'csv' RECORD_DELIMITER = '\\n' SKIP_HEADER = 1 field_optionally_enclosed_by='"' DATE_FORMAT = 'YYYY-MM-DD')'''
    try:
        load_session.sql(copy+form).collect()
        loads_success.append(d)
    except:
        loads_failed.append(d)

Failed to execute query [queryID: 01a9652f-0402-5016-0057-3c03017cda32] copy into place_iq_fact_visit     from @stage_data/synthetic_data_place_iq_fact_visit.csv.gz FILE_FORMAT = (TYPE = 'csv' RECORD_DELIMITER = '\n' SKIP_HEADER = 1 field_optionally_enclosed_by='"' DATE_FORMAT = 'YYYY-MM-DD')
100040 (22007): Date '10/28/2023' is not recognized
  File 'synthetic_data_place_iq_fact_visit.csv.gz', line 2, character 13
  Row 1, column "PLACE_IQ_FACT_VISIT"["PING_DATE":4]
  If you would like to continue loading when an error is encountered, use other values such as 'SKIP_FILE' or 'CONTINUE' for the ON_ERROR option. For more information on loading options, please run 'info loading_data' in a SQL client.
Failed to execute query [queryID: 01a9652f-0402-5016-0057-3c03017cda5a] copy into affinity_txn_data_afs_monthly_master_table_bkup     from @stage_data/synthetic_data_affinity_txn_data_afs_monthly_master_table_bkup.csv.gz FILE_FORMAT = (TYPE = 'csv' RECORD_DELIMITER = '\n' SKIP_HEADER = 1 fiel

In [11]:
loads_failed ### take action

['Synthetic data/synthetic_data_place_iq_fact_visit.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_afs_monthly_master_table_bkup.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_afs_monthly_master_table.csv',
 'Synthetic data/synthetic_data_place_iq_dim_location_14_days.csv',
 'Synthetic data/synthetic_data_prodege_offline_receipts.csv',
 'Synthetic data/synthetic_data_prodege_offline_receipts_categorized.csv',
 'Synthetic data/synthetic_data_place_iq_dim_location.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_afs_monthly_master_table_new_mapping.csv']

In [12]:
loads_success ### validate

['Synthetic data/synthetic_data_place_iq_dim_location_taxonomy_14_days.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_transactions.csv',
 'Synthetic data/synthetic_data_prodege_online_receipts_categorized.csv',
 'Synthetic data/synthetic_data_prodege_amazon_receipts_categorized.csv',
 'Synthetic data/synthetic_data_prodege_online_receipts.csv',
 'Synthetic data/synthetic_data_place_iq_files_metadata.csv',
 'Synthetic data/synthetic_data_place_iq_dim_movement_source.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_brands.csv',
 'Synthetic data/synthetic_data_place_iq_dim_dma.csv',
 'Synthetic data/synthetic_data_prodege_del_ser_receipts.csv',
 'Synthetic data/synthetic_data_place_iq_fact_device_attribute.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_demo_data.csv',
 'Synthetic data/synthetic_data_prodege_member_demos.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_location.csv',
 'Synthetic data/synthetic_data_affinity_txn_data_stores.csv',
 'Synthetic 

In [13]:
loads_still_failed = []

In [14]:
for f in loads_failed:
    df = pd.read_csv(f)
    table_name = f.replace('Synthetic data/synthetic_data_','').split('.')[0]
    try:
        load_session.create_dataframe(df)\
        .write.mode("append")\
        .save_as_table(table_name)
    except:
        loads_still_failed.append(f)

create_temp_table is deprecated. We still respect this parameter when it is True but please consider using `table_type="temporary"` instead.


In [16]:
loads_still_failed

[]

In [17]:
load_session.close()
print('Finished!!!')

Finished!!!
